<a href="https://colab.research.google.com/github/Chandan2403maity/Chandan-Maity_24-14-16/blob/main/VGG16_Accuracy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.utils import to_categorical, Sequence
from tensorflow.keras.datasets import mnist
from sklearn.metrics import classification_report, confusion_matrix
import cv2

# Load dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Convert labels to categorical
y_train_cat = to_categorical(y_train, 10)
y_test_cat = to_categorical(y_test, 10)

# Custom data generator
class MNISTSequence(Sequence):
    def __init__(self, images, labels, batch_size):
        self.images = images
        self.labels = labels
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.images) / self.batch_size))

    def __getitem__(self, idx):
        batch_x = self.images[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.labels[idx * self.batch_size:(idx + 1) * self.batch_size]

        # Resize and normalize
        batch_x = np.array([cv2.resize(img, (224, 224)) for img in batch_x])
        batch_x = np.stack([batch_x] * 3, axis=-1).astype('float32') / 255.0
        return batch_x, batch_y

# Create generators
batch_size = 32
train_seq = MNISTSequence(x_train, y_train_cat, batch_size)
test_seq = MNISTSequence(x_test, y_test_cat, batch_size)

# Load base model
vgg_base = VGG16(weights='imagenet', include_top=False, input_tensor=Input(shape=(224, 224, 3)))

for layer in vgg_base.layers:
    layer.trainable = False

# Custom classifier
x = Flatten()(vgg_base.output)
x = Dense(128, activation='relu')(x)
output = Dense(10, activation='softmax')(x)
model = Model(inputs=vgg_base.input, outputs=output)

# Compile and train
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_seq, epochs=3, validation_data=test_seq)

# Predictions
y_pred_probs = model.predict(test_seq)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = np.argmax(y_test_cat, axis=1)

print("Classification Report:\n", classification_report(y_true, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred))


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
Epoch 1/3


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor']
Received: inputs=Tensor(shape=(None, 224, 224, 3))
  warnings.warn(msg)


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 401s 205ms/step - accuracy: 0.9284 - loss: 0.2680 - val_accuracy: 0.9739 - val_loss: 0.0815
Epoch 2/3
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 387s 206ms/step - accuracy: 0.9870 - loss: 0.0439 - val_accuracy: 0.9868 - val_loss: 0.0438
Epoch 3/3
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 356s 190ms/step - accuracy: 0.9891 - loss: 0.0334 - val_accuracy: 0.9917 - val_loss: 0.0278


/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor']
Received: inputs=Tensor(shape=(32, 224, 224, 3))
  warnings.warn(msg)


313/313 ━━━━━━━━━━━━━━━━━━━━ 52s 165ms/step
Classification Report:
               precision    recall  f1-score   support

           0       0.99      1.00      0.99       980
           1       1.00      1.00      1.00      1135
           2       0.99      0.99      0.99      1032
           3       0.99      1.00      0.99      1010
           4       0.99      1.00      0.99       982
           5       1.00      0.99      0.99       892
           6       1.00      0.99      0.99       958
           7       0.98      0.99      0.99      1028
           8       0.99      0.99      0.99       974
           9       0.99      0.99      0.99      1009

    accuracy                           0.99     10000
   macro avg       0.99      0.99      0.99     10000
weighted avg       0.99      0.99      0.99     10000

Confusion Matrix:
 [[ 976    0    1    0    0    0    1    1    0    1]
 [   0 1131    3    0    0    0    0    1    0    0]
 [   0    0 1018    2    1    0    0    8    2  